In [1]:
# ANN PROJECT

# Muhammad Usama Saleem   19I-1901
# Mohib Hameed            19I-1689
# Taimur Muhammad Khan    19I-1659

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import glob

import tensorflow as tf
from tensorflow import keras

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import math
import random
import scipy.spatial.distance
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

tf.random.set_seed(1000)

In [ ]:
DATA_DIR = tf.keras.utils.get_file(
    "modelnet.zip",
    "http://3dvision.princeton.edu/projects/2014/3DShapeNets/ModelNet10.zip",
    extract=True,
)
directory = os.path.join(os.path.dirname(DATA_DIR), "ModelNet10")

In [ ]:
directory = Path(directory)
classes_all = [dir for dir in sorted(os.listdir(directory)) if os.path.isdir(directory/dir)]
classes_all

['bathtub',
 'bed',
 'chair',
 'desk',
 'dresser',
 'monitor',
 'night_stand',
 'sofa',
 'table',
 'toilet']

In [ ]:
def datapoints_loading(file):
    
    alpha = []
    alpha_2 = []
    alpha_3 = []
    
    beta = []
    beta_2 = []
    beta_3 = []
    
    if 'OFF' != file.readline().strip():
        print('Not a valid OFF header')
    
    number_vertices, number_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    for i_vert in range(number_vertices):
        for s in file.readline().strip().split(' '):
            op_s = float(s)
            alpha.append(op_s)    
    for i in range(len(alpha)):
        alpha_2.append(alpha[i])
        i+=1
        if i % 3 == 0:
            alpha_2 = list(alpha_2)
            alpha_3.append(alpha_2)
            alpha_2 = []
    
    for i_face in range(number_faces):
        for s in file.readline().strip().split(' ')[1:]:
            op_f = int(s)
            beta.append(op_f)
    for i in range(len(beta)):
        beta_2.append(beta[i])
        i+=1
        if i % 3 == 0:
            beta_2 = list(beta_2)
            beta_3.append(beta_2)
            beta_2 = []
            
    return alpha_3, beta_3


In [ ]:
class sampling_points(object):
    
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, point_1, point_2, point_3):
        self.pt_1 = point_1
        self.pt_2 = point_2
        self.pt_3 = point_3
        self.area = 0.5 * ((np.linalg.norm(self.pt_1 - self.pt_2)) + (np.linalg.norm(self.pt_2 - self.pt_3)) + (np.linalg.norm(self.pt_3 - self.pt_1)))
        perimeter = (self.area * (self.area - (np.linalg.norm(self.pt_1 - self.pt_2))) * (self.area - (np.linalg.norm(self.pt_2 - self.pt_3))) * (self.area - (np.linalg.norm(self.pt_3 - self.pt_1))))
        return max(perimeter, 0) ** 0.5

    def sample_point(self, point_1, point_2, point_3):
        self.s, self.t = sorted([random.random(), random.random()])
        self.pt_1 = point_1
        self.pt_2 = point_2
        self.pt_3 = point_3
        self.f = lambda i: self.s * self.pt_1[i] + (self.t-self.s)*self.pt_2[i] + (1-self.t)*self.pt_3[i]
        return (self.f(0), self.f(1), self.f(2))
        
    
    def __call__(self, combo):
        vertices, faces = combo
        vertices = np.array(vertices)
        areas = np.zeros((len(faces)))
        
        for i in range(len(areas)):
          vertex_1 = vertices[faces[i][0]]
          vertex_2 = vertices[faces[i][1]]
          vertex_3 = vertices[faces[i][2]]
          areas[i] = (self.triangle_area(vertex_1, vertex_2, vertex_3))
            
        sampled_vertices = random.choices(faces, weights = areas, cum_weights = None, k = self.output_size)
        sampled_faces = (sampled_vertices)
        s_points = np.zeros((self.output_size, 3))
                        
        for i in range(len(sampled_faces)):
            vertex_4 = vertices[sampled_faces[i][0]]
            vertex_5 = vertices[sampled_faces[i][1]]
            vertex_6 = vertices[sampled_faces[i][2]]
            s_points[i] = (self.sample_point(vertex_4, vertex_5, vertex_6))
        
        return s_points

In [ ]:
class normalization(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        cloud_mean = np.mean(pointcloud, axis=0)
        norm_pointcloud = pointcloud - cloud_mean
        np_mean_cloud = np.max(np.linalg.norm(norm_pointcloud, axis=1))
        cloud_division = norm_pointcloud / np_mean_cloud
        self.norm_pointcloud = cloud_division
        
        return  self.norm_pointcloud

class rotation(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        self.theta = random.random() * 2. * math.pi
        self.alpha = math.cos(self.theta)
        self.beta = -math.sin(self.theta)
        self.charlie = math.sin(self.theta)
        self.delta = math.cos(self.theta)
        self.rot_matrix = np.array([[self.alpha, self.beta, 0],[self.charlie, self.delta, 0],[0, 0, 1]])
        self.dot_product = self.rot_matrix.dot(pointcloud.T).T
        self.rot_pointcloud = self.dot_product
        
        return  self.rot_pointcloud

class noise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        self.shape_1 = pointcloud.shape     
        self.noisy_pointcloud = pointcloud + (np.random.normal(0, 0.02, (self.shape_1)))
        
        return  self.noisy_pointcloud
    
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [ ]:
train_transforms = transforms.Compose([sampling_points(2048), normalization(), rotation(), noise(), ToTensor()])

In [ ]:
class PointCloudData(Dataset):
    
    def __init__(self, valid=False, folder_name = "train", transform = train_transforms):
        self.root_directory = directory
        self.directory = classes_all
        self.transformation = transform
        self.folder_name = folder_name
        self.classes = {self.folder_name: i for i, self.folder_name in enumerate(self.directory)}
        self.transforms = self.transformation if not False else train_transforms
        self.valid = valid
        self.files = []
        self.open_file = ""
        for category in self.classes.keys():
            self.new_dir = self.root_directory/Path(category)/folder_name
            for file in os.listdir(self.new_dir):
                if file.endswith('.off'):
                    self.sample = {}
                    self.name_new = self.new_dir/file
                    self.sample['pcd_path'] = self.name_new
                    self.sample['category'] = category
                    self.files.append(self.sample)
    
    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        self.open_file = file
        self.verts, self.faces = datapoints_loading(self.open_file)
        if self.transforms:
            self.pointcloud = self.transforms((self.verts, self.faces))
        
        return self.pointcloud

    def __getitem__(self, idx):
        self.path_to_file = self.files[idx]['pcd_path']
        self.category = self.files[idx]['category']
        with open(self.path_to_file, 'r') as f:
            self.pointcloud = self.__preproc__(f)
        
        return {'pointcloud': self.pointcloud, 'category': self.classes[self.category]}

In [ ]:
train_ds = PointCloudData(folder_name = "train", transform=train_transforms)
valid_ds = PointCloudData(valid=True, folder_name='test', transform=train_transforms)

In [ ]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10


In [ ]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)

In [ ]:
class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):

      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix

In [ ]:
class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)

        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

In [ ]:
class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64

In [ ]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha=0.0001):
    
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pointnet = PointNet()
optimizer = torch.optim.SGD(pointnet.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
pointnet.to(device)

PointNet(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(64, 64, kernel_size=(1,

In [ ]:
loss_epochs = []
test_acc = []

def train(model, train_loader, val_loader=None,  epochs=15, save=True):
    for epoch in range(epochs):
        pointnet.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            # inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
            optimizer.zero_grad()
            outputs, m3x3, m64x64 = pointnet(inputs.transpose(1,2))

            loss = pointnetloss(outputs, labels, m3x3, m64x64)
            loss.backward()
            optimizer.step()

        loss_epochs.append(running_loss / (i+1))

        pointnet.eval()
        correct = total = 0

        # validation
        if val_loader:
            with torch.no_grad():
                for data in val_loader:
                    inputs, labels = data['pointcloud'].to(device).float(), data['category'].to(device)
                    outputs, __, __ = pointnet(inputs.transpose(1,2))
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
            val_acc = 100. * correct / total
            print('Valid accuracy: %d %%' % val_acc)
        test_acc.append(val_acc)

In [ ]:
train(pointnet, train_loader, valid_loader,  save=False)

[Epoch: 1, Batch:   10 /  125], loss: 2.373
[Epoch: 1, Batch:   20 /  125], loss: 2.247
[Epoch: 1, Batch:   30 /  125], loss: 2.145
[Epoch: 1, Batch:   40 /  125], loss: 2.052
[Epoch: 1, Batch:   50 /  125], loss: 1.961
[Epoch: 1, Batch:   60 /  125], loss: 1.884
[Epoch: 1, Batch:   70 /  125], loss: 1.813
[Epoch: 1, Batch:   80 /  125], loss: 1.738
[Epoch: 1, Batch:   90 /  125], loss: 1.676
[Epoch: 1, Batch:  100 /  125], loss: 1.632
[Epoch: 1, Batch:  110 /  125], loss: 1.590
[Epoch: 1, Batch:  120 /  125], loss: 1.552
Valid accuracy: 51 %
[Epoch: 2, Batch:   10 /  125], loss: 1.084
[Epoch: 2, Batch:   20 /  125], loss: 1.063
[Epoch: 2, Batch:   30 /  125], loss: 1.030
[Epoch: 2, Batch:   40 /  125], loss: 1.000
[Epoch: 2, Batch:   50 /  125], loss: 0.985
[Epoch: 2, Batch:   60 /  125], loss: 0.996
[Epoch: 2, Batch:   70 /  125], loss: 0.972
[Epoch: 2, Batch:   80 /  125], loss: 0.952
[Epoch: 2, Batch:   90 /  125], loss: 0.935
[Epoch: 2, Batch:  100 /  125], loss: 0.928
[Epoch: 2, 